ARE YOU READY! 👏  I'm actually quite excited as this is a project as I have been wanting to do for a while...  Evolutionary Search over Deep Neural Network Architectures 🐣.  This is actually an old concept [1](https://ieeexplore.ieee.org/document/784219/) that has had a bit of a renaissance [2](https://arxiv.org/pdf/1703.00548.pdf) 📄.  Cool packages such as [DEVOL](https://github.com/joeddav/devol) implement a nice version of this for 2D Convoltuion, but I really wanted to do this myself, in order to implement this for other problems like Feed Forward Neural Networks or 1d Convolution 🤓.  [DEAP](http://deap.readthedocs.io/en/master/api/tools.html) is a popular python library for evolutionary computing and seems really versatile, so check it out.  
  
Ok, time to import some libraries...*boring*📚.  

In [31]:
import random
import numpy as np

from deap import base, tools, creator

import tensorflow as tf
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.layers import Input, Dense
from keras import regularizers

import pandas as pd

I recently helped my friend Tim DeWet with a [paper](https://www.biorxiv.org/content/early/2018/06/29/358275) on the use of CRISPR. 🔬 It was very frustrating, but very fun.  I thought... no better dataset to implement a evolving neural network on than on a genetics dataset! I won't get into too much of detail of the data itself. 🍳

In [13]:
data = pd.read_csv('../TimDeWet_Genomics/TrainingData.csv')

In [14]:
down = data.select(lambda col: col.startswith('Down_'), axis=1)

/home/marcussky/.conda/envs/lily/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  """Entry point for launching an IPython kernel.


In [15]:
y_data = data.logRatio
y_transform = y_data.apply(lambda x: (-x+y_data.max()))

So the Genetic Algorithms involves four main steps...  
![Evolution](https://genetic.io/wp-content/uploads/2016/07/processus_en.png)  
  
__Initialize/Selection:__  Where we select new genes.  
__Evalutation:__  Where we evaluate these genes.  
__Cross-over:__  Where we 'breed' out individuals to exchange genes.  
__Mutation:__  Where we introduce new genetic material.  
  
Here, we let the number of neurons and the number of layers in our network be our genes and our loss function is the loss on our hold-out set.  

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

As you can see here, regulatization is really important or the network can get too big, so I included regularization on each layer's regularization for backprop and then added a regularization term for Genetic Algorithm.  

In [74]:
def evalOneMax(individual, alpha=1/50):
    inputs = Input(shape=(down.shape[1],))
    
    dense1 = Dense(abs(individual[0]), 
                   kernel_regularizer=regularizers.l1_l2(0.1, 0.1), 
                   activation='relu')(inputs) if individual[0] != 0 else (inputs)
    
    dense2 = Dense(abs(individual[1]), 
                   kernel_regularizer=regularizers.l1_l2(0.1, 0.1), 
                   activation='relu')(inputs) if individual[1] != 0 else (dense1)
    
    dense3 = Dense(abs(individual[2]), 
                   kernel_regularizer=regularizers.l1_l2(0.1, 0.1), 
                   activation='relu')(inputs) if individual[2] != 0 else (dense2)
    
    output = Dense(1, kernel_regularizer=regularizers.l1_l2(0.1, 0.1), 
                   activation='softmax')(inputs)
    
    model = Model(inputs=inputs, outputs=output)
    
    model.compile('adam', loss=tf.nn.log_poisson_loss)
    model.fit(down.loc[::2,:], y_transform.loc[::2],
                epochs=5,
                batch_size=20,
                shuffle=True,
                validation_split=False,
                verbose=0
           )
    return  (model.evaluate(down.loc[1::2,:], y_transform.loc[1::2], batch_size=128, verbose=0).tolist() + alpha*sum(individual),)

In [75]:
toolbox = base.Toolbox()

N_ATTR = 3

# Attribute generator 
toolbox.register("attr_bool", random.randint,0,64)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, N_ATTR)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

#Evolutionary Strategies
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt,low=0, up=64, indpb=0.3)
toolbox.register("select", tools.selTournament, tournsize=3)

TIME TO EVOLVE!

In [76]:
def main(N_POP=5, CXPB = 0.2, MUTPB = 0.5, GEN=10):
    """
    # N_POP is the population size
    # MUTPB is the probability for mutating an individual
    # CXPB  is the probability with which two individuals are crossed
    # GEN is the number of generations to be run by the algorithm
    """
    
    pop = toolbox.population(N_POP)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit    
    
    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    while max(fits) < 5 and g < GEN:
        # A new generation
        g = g + 1
        print(f"-- Generation {g} --")
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
              
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
              
        pop[:] = offspring
              
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        print(f'Offspring: {offspring}')
        print(f'Fitness Funcitons: {list(map(lambda x: round(x, 3),fits))}')
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        
    return pop[fits.index(min(fits))]

In [77]:
pop = main(N_POP=10, GEN=10)

-- Generation 1 --
Offspring: [[12, 28, 3], [4, 33, 25], [16, 54, 4], [4, 33, 3], [17, 29, 10], [4, 33, 54], [12, 28, 25], [59, 28, 4], [6, 33, 3], [4, 33, 12]]
Fitness Funcitons: [-2.294, -1.913, -1.673, -2.353, -1.413, -2.174, -1.853, -1.334, -2.353, -2.174]
  Min -2.3532354755401608
  Max -1.333711835861206
  Avg -1.9534641904830932
  Std 0.36025956688424743
-- Generation 2 --
Offspring: [[4, 33, 3], [6, 33, 3], [4, 10, 25], [12, 28, 12], [6, 33, 3], [6, 33, 11], [6, 33, 3], [4, 8, 25], [4, 33, 3], [6, 33, 3]]
Fitness Funcitons: [-2.353, -2.353, -2.374, -2.114, -2.353, -2.353, -2.314, -2.414, -2.353, -2.314]
  Min -2.4136019248962404
  Max -2.1135084648132323
  Avg -2.329428523063659
  Std 0.07680434500444087
-- Generation 3 --
Offspring: [[6, 33, 3], [6, 33, 3], [4, 33, 23], [4, 8, 25], [62, 8, 25], [6, 33, 3], [6, 33, 12], [4, 10, 25], [4, 8, 25], [6, 33, 3]]
Fitness Funcitons: [-2.353, -2.353, -2.353, -2.414, -2.414, -2.353, -2.353, -2.374, -2.414, -2.353]
  Min -2.41360192489624

This is awesome! Lets check it out.  What's really interesting is that this network as very few neurons in early and late layers.  That's not something I would have immediately experimented with, as much of the literture suggests the need for many neurons in early layers.  

In [79]:
print(pop)

[3, 26, 3]


In [78]:
loss = evalOneMax(pop, alpha=0)
print(loss)

(-3.152746955871582,)
